In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stunting-balita-detection-121k-rows/data_balita.csv


In [2]:
# membaca dataset dari file CSV
df = pd.read_csv('/kaggle/input/stunting-balita-detection-121k-rows/data_balita.csv')
df

,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm),Status Gizi
0,0,laki-laki,44.591973,stunted
1,0,laki-laki,56.705203,tinggi
2,0,laki-laki,46.863358,normal
3,0,laki-laki,47.508026,normal
4,0,laki-laki,42.743494,severely stunted
...,...,...,...,...
120994,60,perempuan,100.600000,normal
120995,60,perempuan,98.300000,stunted
120996,60,perempuan,121.300000,normal
120997,60,perempuan,112.200000,normal


In [3]:
category_counts = df['Status Gizi'].value_counts()
category_counts

Status Gizi
normal              67755
severely stunted    19869
tinggi              19560
stunted             13815
Name: count, dtype: int64

In [4]:
# mengkodekan jenis kelamin
df['Jenis Kelamin'] = df['Jenis Kelamin'].map({'laki-laki': 0, 'perempuan': 1})

# mengkodekan status gizi
df['Status Gizi'] = df['Status Gizi'].map({'severely stunted': 0, 'stunted': 1, 'normal': 2, 'tinggi': 3})
df

,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm),Status Gizi
0,0,0,44.591973,1
1,0,0,56.705203,3
2,0,0,46.863358,2
3,0,0,47.508026,2
4,0,0,42.743494,0
...,...,...,...,...
120994,60,1,100.600000,2
120995,60,1,98.300000,1
120996,60,1,121.300000,2
120997,60,1,112.200000,2


In [5]:
from sklearn.model_selection import train_test_split

# memisahkan fitur dan target
X = df.iloc[:, 1:6]
X = df.drop('Status Gizi', axis=1)
y = df['Status Gizi']

# membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X

,Umur (bulan),Jenis Kelamin,Tinggi Badan (cm)
0,0,0,44.591973
1,0,0,56.705203
2,0,0,46.863358
3,0,0,47.508026
4,0,0,42.743494
...,...,...,...
120994,60,1,100.600000
120995,60,1,98.300000
120996,60,1,121.300000
120997,60,1,112.200000


In [6]:
from sklearn.neighbors import KNeighborsClassifier

# membuat objek KNN dengan k=3
knn = KNeighborsClassifier(algorithm='kd_tree', n_neighbors=1)

# melatih model KNN dengan data latih
knn.fit(X_train, y_train)


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=1)

In [7]:
from sklearn.metrics import accuracy_score

# membuat prediksi dengan data uji
y_pred = knn.predict(X_test)

# menghitung akurasi model
accuracy = accuracy_score(y_test, y_pred)
print('Akurasi model KNN:', accuracy)


Akurasi model KNN: 0.9990495867768595


In [8]:
from sklearn.metrics import classification_report, confusion_matrix

print('confusion_matrix knn:')
confusion_matrix(y_test, y_pred)

confusion_matrix knn:


array([[ 4129,     1,     0,     0],
       [    6,  2782,     2,     0],
       [    0,     3, 13372,     7],
       [    0,     0,     4,  3894]])

In [9]:
print(classification_report(y_test.values.reshape(-1, 1), y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4130
           1       1.00      1.00      1.00      2790
           2       1.00      1.00      1.00     13382
           3       1.00      1.00      1.00      3898

    accuracy                           1.00     24200
   macro avg       1.00      1.00      1.00     24200
weighted avg       1.00      1.00      1.00     24200



In [10]:
from sklearn.model_selection import cross_val_score

print('Cross val: ', cross_val_score(knn, y_test.values.reshape(-1, 1), y_pred, cv = 5))
print('Accuracy : ', np.mean(cross_val_score(knn, y_test.values.reshape(-1, 1), y_pred)))

Cross val:  [0.99855372 0.99876033 0.99917355 0.99958678 0.99917355]
Accuracy :  0.9990495867768594


In [11]:
# Testing model
test = pd.DataFrame({'Umur (bulan)': [20], 'Jenis Kelamin': [1], 'Tinggi Badan (cm)': [86.2]})

pred = knn.predict(test)
print("Predicted Score = {}".format(np.array(pred)))

Predicted Score = [2]


In [12]:
# Export the model
import joblib

joblib.dump(knn, 'knn.model')

['knn.model']